In [1]:
import tensorflow as tf
import numpy as np
from math import exp
from numpy.random import binomial
from random import shuffle

nx = 5
ny = 5
nz = 5
N = nx*ny*nz
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
# Initializing the neurons
# LSM_neurons = tf.zeros((nx,ny,nz),dtype=tf.float64)

# print(LSM_neurons)

In [3]:
# Storing the IDs of the neurons
LSM_ID = np.zeros((nx,ny,nz),dtype=np.int64)
l = 0
for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            LSM_ID[i,j,k] = l
            l = l + 1

LSM_ID = tf.convert_to_tensor(LSM_ID,dtype=tf.int64)
# print(LSM_ID)

In [4]:
def ID_to_ind(nx,ny,nz,ID):
    x = int(ID/(ny*nz))
    y = int( (ID-(ny*nz)*x) / nz)
    z = int(ID%nz)
    return [x, y, z]

In [37]:
# Storing the synapse connections, and creating the initial weight matrix
k_prob = 1
r_sq = 1**2

W_init = 100 # initial weight
Weights = np.zeros((N,N))

N_in = int(N*0.8)
neuron_type = [ int(i<N_in) for i in range(N)]
shuffle(neuron_type) # 1 for excitatory, 0 for inhibitory

synapes = [dict() for i in range(N)]    # an array of dictonaries which store the location of neuron, type of neuron, and the IDs of the neurons it is connected to

for l in range(N):
    loc = ID_to_ind(nx,ny,nz,l)
    n_type = neuron_type[l]
    cons = []
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                if l != int(LSM_ID[i,j,k]):
                    dist_sq = (loc[0]-i)**2 + (loc[1]-j)**2 + (loc[2]-k)**2
                    probability = k_prob* exp(-1*dist_sq/r_sq)
                    check = binomial(1,probability)
                    if check == 1:
                        cons.append(int(LSM_ID[i,j,k]))
                        Weights[l,int(LSM_ID[i,j,k])] = W_init    
    synapes[l] = {"Location":loc, "Neuron_type":n_type, "connections":cons}

Weights = tf.convert_to_tensor(Weights,dtype=tf.float64)

In [ ]:
from brian2 import *

#### Simplified model in paper(IEEE ZHANG et al)
Effect of Synaptic models **Section V(A), eqns -- 19, 20, 21**

In [ ]:
from exceptions import ValueError
global vrest, vth, t_refrac
vrest, vth, t_refrac = -70, 20, 2

#### LIF Neuron single step solver
Converted from MATLAB.  Modified version of LIF solver for HW1, with N neurons and refractory period added, solves just a single timestep 

In [111]:
temp = np.array([i for i in range(N)])
temp[5:N] = 1
temp

array([0, 1, 2, 3, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [70]:
params = {'C':300, 'g_L':30, 'E_L':-70, 'V_T':20, 'R_p':2}
C, g_L, E_L, V_T, R_p = params.values()
def LIF(V_neuron_prev,I_input_prev,I_input_next,N,h,index_next,index_prev_spike, params):
    C, g_L, E_L, V_T, R_p = params.values()
    R_p_ind = tf.math.ceil(R_p/h)
    
    V_neuron_next = tf.math.scalar_mul(E_L,tf.ones((N,), dtype='float'))
    Spike_next = tf.zeros((N,), dtype='float')
    
    k1 = (1/C)*(-g_L*(V_neuron_prev-E_L)+I_input_prev)
    V_temp = V_neuron_prev + k1*h/2
    I_temp = I_input_prev/2 + I_input_next/2
    k2 = (1/C)*(-g_L*(V_temp-E_L)+I_temp)
    V_temp = V_neuron_prev + k2*h
    
    for i in range(N):
        if index_next-int(index_prev_spike[i]) < R_p_ind:
            V_neuron_next = tf.tensor_scatter_nd_update(V_neuron_next,[[i]],[[E_L]])
        elif np.float64(V_temp[i]) < V_T:
            V_neuron_next = tf.tensor_scatter_nd_update(V_neuron_next,[[i]],[[np.float64(V_temp[i])]])
        else:
            Spike_next    = tf.tensor_scatter_nd_update(V_neuron_next,[[i]],[[int(1)]]) 
            V_neuron_next = tf.tensor_scatter_nd_update(V_neuron_next,[[i]],[[np.float64(V_temp[i])]])
    
    return V_neuron_next, Spike_next
    
            

In [ ]:
# def syn_res(syn_string, time_ind, spike_timing_mat, h, type_syn=None):
def syn_res(syn_string,type_syn,t,time,i,j,w_ij,del_i,h,M):  # spike in neuron i, produces a synaptic current in neuron j, weight = w_ij
    syn_curr = np.zeros((M,))

    ts_ds = np.float64(time[t]) + del_i
    ind = int(np.where(time == ts_ds))
    if syn_string == "static":
        syn_curr[ind] = 1/h
        # result_mat = [1 if time_ind == a_spike else 0 for a_spike in spike_timing_mat]
        # return result_mat
    elif syn_string == "first-order":
        tau_s = 4 * h # Given in paper
        syn_curr[ind:M] = w_ij * (1/tau_s) * np.exp(-(1/tau_s)*(time[ind:M] -ts_ds))
        # result_mat = (1/tau_s) * (np.exp(- (time_ind - spike_timing_mat)/tau_s) 
        # * np.heaviside(time_ind - spike_timing_mat))
        # return result_mat
    elif syn_string == "second-order":
        if type_syn is None:
            raise ValueError('type required')
        elif type_syn == 1:
            tau_s1, tau_s2 = 4, 8
        elif type_syn == 0:
            tau_s1, tau_s2 = 4, 2
        else:
            raise ValueError('type invalid')
        
        syn_curr[ind:M] = (w_ij/(tau_s1-tau_s2)) * (np.exp(-(1/tau_s1)*(time[ind:M] -ts_ds)) -np.exp(-(1/tau_s2)*(time[ind:M] -ts_ds)))
        # result_mat = ((1/(tau_s1 - tau_s2)) * np.exp(- (time_ind - spike_timing_mat)/tau_s1)
        #               * np.heaviside(time_ind - spike_timing_mat)
        #               - ((1/(tau_s1 - tau_s2)) * np.exp(- (time_ind - spike_timing_mat)/tau_s2)
        #               * np.heaviside(time_ind - spike_timing_mat)))
        # return result_mat
            
    return syn_curr

#### Reservoir solver
Converted from **MATLAB assignment 3 Q2** neuron solver <br>

In [ ]:
def reservoir_solver(nx,ny,nz, synapes, M, h, I_app,threshold, **kwargs):
    global vrest, vth
    N = nx*ny*nz
    I_syn = tf.zeros((N,M))
    I_total = tf.zeros((N,M))
    V_neurons = vrest*tf.ones((N,M))
    Spikes = tf.zeros((N,M))
    Calcium_conc = tf.zeros((N,M))
    weight_mat = tf.ones((N,M))

    tau_m, syn_string, h = 32, "static", 0.5
    
    index_prev_spike = -1*(M)*tf.ones((N,))

    time = np.array([j*h for j in range(M)])

    for t in range(1,M):
        I_total = I_app + I_syn
        # for j in range(N):
            # V_neuron[j,i] = LIF_rsp_one(j, Weight_mat[j], V_neuron[j-1,i], Connect_mat[j], syn_string,
            #                            tau_m, Spikes, Delay_mat, I_total[:,i-1], h, time_ind=i,
            #                            Calcium_conc = Calcium_conc)
        V_neuron, Spike = LIF(V_neurons[:,t-1],I_total[:,t-1],I_total[:,t],N,h,t,index_prev_spike, params)
        V_neurons[:,t] = V_neuron
        Spikes[:,t] = Spike
        
        for i in range(N):
            if Spike[i] == 1:
                index_prev_spike[i] = t

                I_syn_additional = tf.zeros((N,M))
                neurons = synapes[i]["connections"]
                neuron_tp = synapes[i]["Neuron_type"]
                for j in range(len(neurons)):
                    I_syn_additional[neurons[j],t:M] = syn_res(syn_string,neuron_tp,t,time,i,neurons[j],Weights[i,j],Delay[i],h,M)  # need to create Delay tensor
      
        I_syn = I_syn + I_syn_additional

    
    return V_neurons, Spikes
    

Models for implementing proposed learning rule **Section IV(c), eqns -- 12, 13, 14**

In [11]:
def LIF_rsp_one(neuron_id,Weight_mat, Vm_prev, FanIn_list, syn_string, 
                tau_m, spikeTime_mat, Delay_mat, it_prev, h, time_ind, **kwargs):
    global vrest, vth
    spike_happened = 0
    V_temp = Vm_prev  - Vm_prev/tau_m + it_prev
    for i in FanIn_list:
        V_temp += Weight_mat[i] * syn_res(syn_string, time_ind, spikeTime_mat[:, i] + Delay_mat[i], h) 
    if time_ind - spikeTime_mat[neuron_id][-1] <= tf.math.ceil(t_refrac/h):
        V_temp = vrest
    elif V_temp > vth:
        spikeTime_mat[time_ind, neuron_id] = 1
        V_temp = vrest
    else:
        pass
    return V_temp
    
    

Weight learning by calculation of Digitized Calcium concentration updation, **Equations 15 , 16 in IEEE paper**<br>
Incomplete, have to read section **III(A), III(B)**

In [12]:
def Weight_learner(tau_c, last_conc, time_ind, this_neuron_spiked):
    new_conc = last_conc - last_conc/tau_c + np.sum(np.heaviside(time_ind - this_neuron_spiked))
    ## Need to implement the learning rule here 
    return new_conc
    